In [1]:
import warnings
import sys
sys.path.insert(0, '../src')

In [2]:
import re
import glob
from collections import Counter

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

In [3]:
from felix_ml_tools import macgyver as mg
from utils import *
import config

In [4]:
warnings.filterwarnings('ignore')

pd.set_option("max_columns", None)
pd.set_option("max_rows", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline 

# Ingestão de datasets (ETL)

**CBOs**

In [5]:
# https://www.hardware.com.br/comunidade/attachments/cbo-xlsx.37816/
cbos = pd.read_csv('../data/cbo.csv')

inspect(cbos);

shape: (2428, 2)
columns: ['codigoCbo', 'ocupacaoCbo']


,codigoCbo,ocupacaoCbo
0,848505,ABATEDOR
1,764305,ACABADOR DE CALCADOS
2,766305,ACABADOR DE EMBALAGENS (FLEXIVEIS E CARTOTECNI...
3,716105,ACABADOR DE SUPERFICIES DE CONCRETO
4,848510,ACOUGUEIRO


**CID-10**

In [6]:
# TODO

**CNAE**

In [7]:
# mapeamento de CNAEs (RFB) e suas divisoes

rfb_cnae = (
    pd.read_parquet('../data/rfb_cnae.parquet')
    .assign(codigoDivisaoCnae=lambda x: x['codigoCnae'].str.slice(0, 2))
)

# https://concla.ibge.gov.br/busca-online-cnae.html?view=estrutura
divisao_cnaes_str = '''01 .. 03 AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA
05 .. 09 INDÚSTRIAS EXTRATIVAS
10 .. 33 INDÚSTRIAS DE TRANSFORMAÇÃO
35 .. 35 ELETRICIDADE E GÁS
36 .. 39 ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO
41 .. 43 CONSTRUÇÃO
45 .. 47 COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS
49 .. 53 TRANSPORTE, ARMAZENAGEM E CORREIO
55 .. 56 ALOJAMENTO E ALIMENTAÇÃO
58 .. 63 INFORMAÇÃO E COMUNICAÇÃO
64 .. 66 ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS
68 .. 68 ATIVIDADES IMOBILIÁRIAS
69 .. 75 ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS
77 .. 82 ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEMENTARES
84 .. 84 ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL
85 .. 85 EDUCAÇÃO
86 .. 88 SAÚDE HUMANA E SERVIÇOS SOCIAIS
90 .. 93 ARTES, CULTURA, ESPORTE E RECREAÇÃO
94 .. 96 OUTRAS ATIVIDADES DE SERVIÇOS
97 .. 97 SERVIÇOS DOMÉSTICOS
99 .. 99 ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS
'''

divisao_cnaes_map = {
    'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA': (1, 3),
    'INDÚSTRIAS EXTRATIVAS': (5, 9),
    'INDÚSTRIAS DE TRANSFORMAÇÃO': (10, 33),
    'ELETRICIDADE E GÁS': (35, 35),
    'ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO': (36, 39),
    'CONSTRUÇÃO': (41, 43),
    'COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS': (45, 47),
    'TRANSPORTE, ARMAZENAGEM E CORREIO': (49, 53),
    'ALOJAMENTO E ALIMENTAÇÃO': (55, 56),
    'INFORMAÇÃO E COMUNICAÇÃO': (58, 63),
    'ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS': (64, 66),
    'ATIVIDADES IMOBILIÁRIAS': (68, 68),
    'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS': (69, 75),
    'ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEMENTARES': (77, 82),
    'ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL': (84, 84),
    'EDUCAÇÃO': (85, 85),
    'SAÚDE HUMANA E SERVIÇOS SOCIAIS': (86, 88),
    'ARTES, CULTURA, ESPORTE E RECREAÇÃO': (90, 93),
    'ATIVIDADES DE ORGANIZAÇÕES ASSOCIATIVAS': (94, 94),
    'OUTRAS ATIVIDADES DE SERVIÇOS': (95, 96),
    'SERVIÇOS DOMÉSTICOS': (97, 97),
    'ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS': (99, 99),
}

divisao_alias_map = {
    'AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA': 'AGROPECU',
    'INDÚSTRIAS EXTRATIVAS': 'INDEXTRA',
    'INDÚSTRIAS DE TRANSFORMAÇÃO': 'INDTRANS',
    'ELETRICIDADE E GÁS': 'ELETRGAS',
    'ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO': 'AGUAESGO',
    'CONSTRUÇÃO': 'CONSTRUC',
    'COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS': 'AUTOMOTO',
    'TRANSPORTE, ARMAZENAGEM E CORREIO': 'TRANSPOR',
    'ALOJAMENTO E ALIMENTAÇÃO': 'ALOJALIM',
    'INFORMAÇÃO E COMUNICAÇÃO': 'INFOCOMU',
    'ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS': 'FINANSEG',
    'ATIVIDADES IMOBILIÁRIAS': 'IMOBILIA',
    'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS': 'PROCIENT',
    'ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEMENTARES': 'ADMICOMP',
    'ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL': 'PUBLSOCI',
    'EDUCAÇÃO': 'EDUCACAO',
    'SAÚDE HUMANA E SERVIÇOS SOCIAIS': 'SAUDSOCI',
    'ARTES, CULTURA, ESPORTE E RECREAÇÃO': 'ARTCULES',
    'ATIVIDADES DE ORGANIZAÇÕES ASSOCIATIVAS': 'ONGASSOC',
    'OUTRAS ATIVIDADES DE SERVIÇOS': 'OUTRSERV',
    'SERVIÇOS DOMÉSTICOS': 'DOMESTIC',
    'ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS': 'INTERNAC',
}

# https://stackoverflow.com/a/50098393
divisao_cnaes_criteria = [rfb_cnae['codigoDivisaoCnae'].astype(int).between(v[0], v[1]) for v in divisao_cnaes_map.values()]
divisao_cnaes_values = list(divisao_cnaes_map.keys())
rfb_cnae['divisaoCnae'] = np.select(divisao_cnaes_criteria, divisao_cnaes_values, 0)
rfb_cnae['divisaoCnaeAlias'] = rfb_cnae['divisaoCnae'].map(divisao_alias_map).str.lower()

rfb_cnae_divisao = rfb_cnae[['codigoDivisaoCnae', 'divisaoCnae', 'divisaoCnaeAlias']].drop_duplicates()

inspect(rfb_cnae);

inspect(rfb_cnae_divisao);

shape: (1360, 5)
columns: ['codigoCnae', 'descricaoCnae', 'codigoDivisaoCnae', 'divisaoCnae', 'divisaoCnaeAlias']


,codigoCnae,descricaoCnae,codigoDivisaoCnae,divisaoCnae,divisaoCnaeAlias
0,2865800,FABRICACAO DE MAQUINAS E EQUIPAMENTOS PARA AS ...,28,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans
1,3511501,GERACAO DE ENERGIA ELETRICA,35,ELETRICIDADE E GÁS,eletrgas
2,3530100,"PRODUCAO E DISTRIBUICAO DE VAPOR, AGUA QUENTE ...",35,ELETRICIDADE E GÁS,eletrgas
3,2223400,FABRICACAO DE TUBOS E ACESSORIOS DE MATERIAL P...,22,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans
4,2532201,PRODUCAO DE ARTEFATOS ESTAMPADOS DE METAL,25,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans


shape: (87, 3)
columns: ['codigoDivisaoCnae', 'divisaoCnae', 'divisaoCnaeAlias']


,codigoDivisaoCnae,divisaoCnae,divisaoCnaeAlias
0,28,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans
1,35,ELETRICIDADE E GÁS,eletrgas
3,22,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans
4,25,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans
6,32,INDÚSTRIAS DE TRANSFORMAÇÃO,indtrans


**Municípios do IBGE**

In [8]:
ibge_municipio = pd.read_parquet('../data/ibge_municipio_pib.parquet')

ibge_municipio['ufCodigo'] = ibge_municipio['ufCodigo'].astype(str)
ibge_municipio['municipioCodigo'] = ibge_municipio['municipioCodigo'].astype(str)
ibge_municipio['municipioCodigo6d'] = ibge_municipio['municipioCodigo'].str.slice(0, 6)

ibge_municipio['nomeMunicipioNorm'] = ibge_municipio['nomeMunicipio'].apply(replace_accents)
ibge_municipio['nomeMunicipioNorm'] = ibge_municipio['nomeMunicipioNorm'].apply(lambda x: re.sub(r"[^a-zA-Z\d\_]+", " ", x))
ibge_municipio['nomeMunicipioNorm'] = ibge_municipio['nomeMunicipioNorm'].str.upper()

ibge_municipio = ibge_municipio[['uf', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'nomeMunicipioNorm', 'municipioCodigo', 'municipioCodigo6d']]
ibge_municipio = ibge_municipio.drop_duplicates()

inspect(ibge_municipio);

shape: (5570, 7)
columns: ['uf', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'nomeMunicipioNorm', 'municipioCodigo', 'municipioCodigo6d']


,uf,ufCodigo,nomeUf,nomeMunicipio,nomeMunicipioNorm,municipioCodigo,municipioCodigo6d
0,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,1100015,110001
1,RO,11,Rondônia,Ariquemes,ARIQUEMES,1100023,110002
2,RO,11,Rondônia,Cabixi,CABIXI,1100031,110003
3,RO,11,Rondônia,Cacoal,CACOAL,1100049,110004
4,RO,11,Rondônia,Cerejeiras,CEREJEIRAS,1100056,110005


**Município - Total de pessoas com empregado formal (RAIS/CAGED)**

In [9]:
rais_caged_cnpj = inspect(
    pd.read_parquet('../data/rais_caged_cnpj.parquet')
    .rename(columns={'enderecoUf': 'uf', 'enderecoMunicipio': 'nomeMunicipioNorm'})
    .merge(ibge_municipio, on=['uf', 'nomeMunicipioNorm'])
)

rais_caged_municipio_funcionarios = (
    rais_caged_cnpj
    .groupby(['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'ano'])
    .agg(
        totalFuncionarios=('funcionarios', 'sum'),
    )
    .reset_index()
    #.rename(columns={'uf': 'uf'})
)

inspect(rais_caged_municipio_funcionarios);

shape: (22832693, 10)
columns: ['cnpj', 'ano', 'funcionarios', 'uf', 'nomeMunicipioNorm', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d']


,cnpj,ano,funcionarios,uf,nomeMunicipioNorm,ufCodigo,nomeUf,nomeMunicipio,municipioCodigo,municipioCodigo6d
0,00000000066370,2010,6,SP,SAO PAULO,35,São Paulo,São Paulo,3550308,355030
1,00000000066370,2011,7,SP,SAO PAULO,35,São Paulo,São Paulo,3550308,355030
2,00000000066370,2012,8,SP,SAO PAULO,35,São Paulo,São Paulo,3550308,355030
3,00000000066370,2013,9,SP,SAO PAULO,35,São Paulo,São Paulo,3550308,355030
4,00000000066370,2014,11,SP,SAO PAULO,35,São Paulo,São Paulo,3550308,355030


shape: (60515, 7)
columns: ['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'ano', 'totalFuncionarios']


,uf,ufCodigo,nomeMunicipio,municipioCodigo,municipioCodigo6d,ano,totalFuncionarios
0,AC,12,Acrelândia,1200013,120001,2010,232
1,AC,12,Acrelândia,1200013,120001,2011,264
2,AC,12,Acrelândia,1200013,120001,2012,283
3,AC,12,Acrelândia,1200013,120001,2013,310
4,AC,12,Acrelândia,1200013,120001,2014,364


**Município CNAEs**

In [10]:
# Lendo cnaes de cada CNPJ ativo antes da pandemia (2020)
rfb_cnpj_cnaes_subpaths = glob.glob('../data/rfb_cnpj_cnaes.parquet/rfb_cnpj_cnaes.parquet/*.parquet')
rfb_cnpj_cnaes_subdfs = []

for subpath in rfb_cnpj_cnaes_subpaths:
    subdf = pd.read_parquet(subpath)
    subdf['abertura'] = pd.to_datetime(subdf['abertura'], errors = 'coerce')
    subdf['aberturaAno'] = subdf['abertura'].dt.year

    subdf['situacaoCadastralData'] = pd.to_datetime(subdf['situacaoCadastralData'], errors = 'coerce')
    subdf['situacaoCadastralAno'] = subdf['situacaoCadastralData'].dt.year

    # nos temos dados de homicidios ate 2018,
    # entao vamos considerar apenas empresas do norteste que estavam ativas antes da pandemia 2020
    # - empresas ativas
    # - empresas baixadas em 2020, 2021
    # - empresas abertas ate 2020
    subdf = subdf[
        subdf['enderecoUf'].isin(config.target_ufs)
        & (subdf['abertura'].dt.year < 2020)
        & (
            (subdf['situacaoCadastralStatus'].str.contains('BAIXADA') == False)
            |  (
                subdf['situacaoCadastralStatus'].str.contains('BAIXADA')
                & (subdf['situacaoCadastralData'].dt.year >= 2020)
            )
        )
    ]

    rfb_cnpj_cnaes_subdfs.append(subdf)

rfb_cnpj_cnaes = pd.concat(rfb_cnpj_cnaes_subdfs)

print(Counter(rfb_cnpj_cnaes['enderecoUf']))

inspect(rfb_cnpj_cnaes);

Counter({'BA': 1211592, 'PE': 696226, 'CE': 686053, 'MA': 356836, 'PB': 294190, 'RN': 278747, 'AL': 213435, 'PI': 213406, 'SE': 145610})
shape: (4096095, 10)
columns: ['cnpj', 'cnaes', 'enderecoUf', 'enderecoMunicipio', 'abertura', 'situacaoCadastralData', 'situacaoCadastralStatus', 'porte', 'aberturaAno', 'situacaoCadastralAno']


,cnpj,cnaes,enderecoUf,enderecoMunicipio,abertura,situacaoCadastralData,situacaoCadastralStatus,porte,aberturaAno,situacaoCadastralAno
2,00075886000132,"[4732600, 4731800]",PI,FLORIANO,1994-05-26,2020-08-25,ATIVA,ME,1994.000,2020.000
3,00076953000133,"[8730199, 8711505, 8720499]",PI,COIVARAS,1994-05-25,2018-11-07,INAPTA,DEMAIS,1994.000,2018.000
6,00148449000100,"[4530705, 4763603, 4530703, 4789099, 4541206]",PI,LUZILANDIA,1994-08-17,2005-11-03,ATIVA,ME,1994.000,2005.000
9,00232265000115,[4784900],PI,JARDIM DO MULATO,1994-10-06,2018-09-11,INAPTA,ME,1994.000,2018.000
10,00288485000160,[5611203],PI,GILBUES,1994-11-10,2019-01-29,INAPTA,DEMAIS,1994.000,2019.000


In [11]:
# mapeamento de CNAEs para divisoes

rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes[['cnpj', 'enderecoUf', 'enderecoMunicipio', 'cnaes']]
rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes_divisao.explode('cnaes')
rfb_cnpj_cnaes_divisao['cnaes'] = rfb_cnpj_cnaes_divisao['cnaes'].str.slice(0,2)
rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes_divisao.drop_duplicates()
rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes_divisao.rename(columns={'cnaes': 'codigoDivisaoCnae'})
rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes_divisao.merge(rfb_cnae_divisao, on=['codigoDivisaoCnae'])

rfb_cnpj_cnaes_divisao = (
    rfb_cnpj_cnaes_divisao
    .groupby(['cnpj', 'enderecoUf', 'enderecoMunicipio', 'divisaoCnae', 'divisaoCnaeAlias'])
    .agg(codigoDivisaoCnaes=('codigoDivisaoCnae', list))
    .reset_index()
    .rename(columns={'enderecoUf': 'uf', 'enderecoMunicipio': 'nomeMunicipioNorm'})
    .merge(ibge_municipio, on=['uf', 'nomeMunicipioNorm'])
)

inspect(rfb_cnpj_cnaes_divisao);

shape: (5916177, 11)
columns: ['cnpj', 'uf', 'nomeMunicipioNorm', 'divisaoCnae', 'divisaoCnaeAlias', 'codigoDivisaoCnaes', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d']


,cnpj,uf,nomeMunicipioNorm,divisaoCnae,divisaoCnaeAlias,codigoDivisaoCnaes,ufCodigo,nomeUf,nomeMunicipio,municipioCodigo,municipioCodigo6d
0,00000000000604,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740
1,00000000034681,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740
2,00000000091057,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740
3,00000000095710,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740
4,00000000126888,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740


In [12]:
# identificando empresas publicas

rfb_cnpj_publico = (
    (rfb_cnpj_cnaes_divisao
    .assign(publico=lambda x: x['divisaoCnae'] == 'ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL'))
    [['cnpj', 'publico']]
    .drop_duplicates()
)
rfb_cnpj_publico = rfb_cnpj_publico[rfb_cnpj_publico['publico'] == True]

rfb_cnpj_cnaes_divisao = rfb_cnpj_cnaes_divisao.merge(rfb_cnpj_publico, on=['cnpj'], how='left')
rfb_cnpj_cnaes_divisao['publico'] = rfb_cnpj_cnaes_divisao['publico'].fillna(False)

inspect(rfb_cnpj_cnaes_divisao[rfb_cnpj_cnaes_divisao['publico'] == True]);

shape: (26724, 12)
columns: ['cnpj', 'uf', 'nomeMunicipioNorm', 'divisaoCnae', 'divisaoCnaeAlias', 'codigoDivisaoCnaes', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'publico']


,cnpj,uf,nomeMunicipioNorm,divisaoCnae,divisaoCnaeAlias,codigoDivisaoCnaes,ufCodigo,nomeUf,nomeMunicipio,municipioCodigo,municipioCodigo6d,publico
69,00038166000873,BA,SALVADOR,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",publsoci,[84],29,Bahia,Salvador,2927408,292740,True
70,00038166000873,BA,SALVADOR,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,[64],29,Bahia,Salvador,2927408,292740,True
3259,00375972000756,BA,SALVADOR,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",publsoci,[84],29,Bahia,Salvador,2927408,292740,True
3297,00381056000800,BA,SALVADOR,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",publsoci,[84],29,Bahia,Salvador,2927408,292740,True
3412,00394429001425,BA,SALVADOR,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",publsoci,[84],29,Bahia,Salvador,2927408,292740,True


In [13]:
# agregando empresas por municipio e divisao economica

rfb_municipio_cnaes = (
    rfb_cnpj_cnaes_divisao
    .groupby(['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'divisaoCnae', 'divisaoCnaeAlias'])
    .agg(
        qtEmpresas=('cnpj', 'count'),
        qtEmpresasPublicas=('publico', lambda x: x.astype(int).sum()),
    )
    .reset_index()
)

rfb_municipio_cnaes[
    (rfb_municipio_cnaes['uf'] == 'PB')
    & (rfb_municipio_cnaes['nomeMunicipio'] == 'Esperança')
]

,uf,ufCodigo,nomeMunicipio,municipioCodigo,municipioCodigo6d,divisaoCnae,divisaoCnaeAlias,qtEmpresas,qtEmpresasPublicas
19270,PB,25,Esperança,2506004,250600,"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL",publsoci,14,14
19271,PB,25,Esperança,2506004,250600,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",agropecu,21,0
19272,PB,25,Esperança,2506004,250600,ALOJAMENTO E ALIMENTAÇÃO,alojalim,172,0
19273,PB,25,Esperança,2506004,250600,"ARTES, CULTURA, ESPORTE E RECREAÇÃO",artcules,33,0
19274,PB,25,Esperança,2506004,250600,ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEME...,admicomp,154,0
19275,PB,25,Esperança,2506004,250600,ATIVIDADES DE ORGANIZAÇÕES ASSOCIATIVAS,ongassoc,141,1
19276,PB,25,Esperança,2506004,250600,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",finanseg,19,0
19277,PB,25,Esperança,2506004,250600,ATIVIDADES IMOBILIÁRIAS,imobilia,27,0
19278,PB,25,Esperança,2506004,250600,"ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",procient,136,0
19279,PB,25,Esperança,2506004,250600,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,automoto,1689,0


**Município - Rendimento Escolar (INEP)**

In [14]:
inep_municipio_rendimento_escolar = pd.read_csv('../data/7. INEP/taxa_de_rendimento/TX_REND_MUNICIPIOS.csv')

inep_municipio_rendimento_escolar.columns = inep_municipio_rendimento_escolar.columns.str.lower()

print(inep_municipio_rendimento_escolar['ano'].unique())

inep_municipio_rendimento_escolar = inep_municipio_rendimento_escolar[
    #(inep_municipio_rendimento_escolar['ano'] == 2018)
    (inep_municipio_rendimento_escolar['dependad'] == 'Pública') # nosso foco aqui serão escolas públicas
    & (inep_municipio_rendimento_escolar['tipoloca'] == 'Total')  # escolas em zona rural or urbana
]

inep_municipio_rendimento_escolar_cols = [
    'aprv_ef_total',
    'reprv_ef_total',
    'abd_ef_total',
    
    'aprv_ef_anos_iniciais',
    'reprv_ef_anos_iniciais',
    'aprv_ef_anos_finais',
    'reprv_ef_anos_finais',

    'aprv_em_total',
    'reprv_em_total',
    'abd_em_total',

    'aprv_em_n_seriado',
    'reprv_em_n_seriado',
    'abd_em_n_seriado',
]

inep_municipio_rendimento_escolar = inep_municipio_rendimento_escolar.drop(columns=['dependad', 'tipoloca'])

inep_municipio_rendimento_escolar = inep_municipio_rendimento_escolar[['cod_municipio', 'ano'] +  inep_municipio_rendimento_escolar_cols]

inep_municipio_rendimento_escolar[inep_municipio_rendimento_escolar_cols] = inep_municipio_rendimento_escolar[inep_municipio_rendimento_escolar_cols].apply(lambda x: x.str.replace(',', '.').astype(float))

inep_municipio_rendimento_escolar = (
    inep_municipio_rendimento_escolar
    .rename(columns={'cod_municipio': 'municipioCodigo'})
    .assign(municipioCodigo=lambda x: x['municipioCodigo'].astype(str))
    .merge(ibge_municipio, on=['municipioCodigo'])
)

inep_municipio_rendimento_escolar.columns = [underscore_to_camelcase(c) for c in inep_municipio_rendimento_escolar.columns]

inspect(inep_municipio_rendimento_escolar);

[2013 2014 2015 2016 2017 2018 2019]
shape: (27850, 21)
columns: ['municipioCodigo', 'ano', 'aprvEfTotal', 'reprvEfTotal', 'abdEfTotal', 'aprvEfAnosIniciais', 'reprvEfAnosIniciais', 'aprvEfAnosFinais', 'reprvEfAnosFinais', 'aprvEmTotal', 'reprvEmTotal', 'abdEmTotal', 'aprvEmNSeriado', 'reprvEmNSeriado', 'abdEmNSeriado', 'uf', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'nomeMunicipioNorm', 'municipioCodigo6d']


,municipioCodigo,ano,aprvEfTotal,reprvEfTotal,abdEfTotal,aprvEfAnosIniciais,reprvEfAnosIniciais,aprvEfAnosFinais,reprvEfAnosFinais,aprvEmTotal,reprvEmTotal,abdEmTotal,aprvEmNSeriado,reprvEmNSeriado,abdEmNSeriado,uf,ufCodigo,nomeUf,nomeMunicipio,nomeMunicipioNorm,municipioCodigo6d
0,1100015,2015,91.800,6.000,2.200,95.400,3.900,85.800,9.500,85.500,3.300,11.200,NaN,NaN,NaN,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,110001
1,1100015,2016,90.400,6.500,3.100,92.000,6.900,88.100,5.800,88.700,0.700,10.600,NaN,NaN,NaN,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,110001
2,1100015,2017,93.000,5.600,1.400,94.300,5.200,91.000,6.200,88.100,4.700,7.200,NaN,NaN,NaN,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,110001
3,1100015,2018,90.800,7.400,1.800,92.600,7.100,88.200,7.800,90.600,2.600,6.800,NaN,NaN,NaN,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,110001
4,1100015,2019,94.000,4.400,1.600,94.600,4.900,93.200,3.800,92.200,2.000,5.800,NaN,NaN,NaN,RO,11,Rondônia,Alta Floresta D'Oeste,ALTA FLORESTA D OESTE,110001


**Município - PIB (IBGE)**

In [15]:
ibge_municipio_pib = pd.read_parquet('../data/ibge_municipio_pib.parquet')
ibge_municipio_pib['ufCodigo'] = ibge_municipio_pib['ufCodigo'].astype(str)
ibge_municipio_pib['municipioCodigo'] = ibge_municipio_pib['municipioCodigo'].astype(str)
ibge_municipio_pib['municipioCodigo6d'] = ibge_municipio_pib['municipioCodigo'].str.slice(0, 6)

inspect(ibge_municipio_pib);

shape: (5570, 18)
columns: ['uf', 'ufCodigo', 'nomeUf', 'nomeMunicipio', 'municipioCodigo', 'valorBrutoAgropecuraria1000', 'valorBrutoIndustria1000', 'valorBrutoServicos1000', 'valorBrutoUtilidade1000', 'valorBruto1000', 'impostosProdutos1000', 'pib1000', 'pibPerCapita', 'atividadeMaisRelevanteValorBruto1', 'atividadeMaisRelevanteValorBruto2', 'atividadeMaisRelevanteValorBruto3', 'ano', 'municipioCodigo6d']


,uf,ufCodigo,nomeUf,nomeMunicipio,municipioCodigo,valorBrutoAgropecuraria1000,valorBrutoIndustria1000,valorBrutoServicos1000,valorBrutoUtilidade1000,valorBruto1000,impostosProdutos1000,pib1000,pibPerCapita,atividadeMaisRelevanteValorBruto1,atividadeMaisRelevanteValorBruto2,atividadeMaisRelevanteValorBruto3,ano,municipioCodigo6d
0,RO,11,Rondônia,Alta Floresta D'Oeste,1100015,169623,27805,117717,154926,470072,28793,498864,19612,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária",Demais serviços\r,2017,110001
1,RO,11,Rondônia,Ariquemes,1100023,165217,330319,933935,634591,2064062,232012,2296074,21390,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,2017,110002
2,RO,11,Rondônia,Cabixi,1100031,63022,4797,22431,43150,133399,5494,138894,22316,"Administração, defesa, educação e saúde públic...","Pecuária, inclusive apoio à pecuária","Agricultura, inclusive apoio à agricultura e a...",2017,110003
3,RO,11,Rondônia,Cacoal,1100049,255060,224595,908665,521368,1909689,217835,2127523,24038,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,2017,110004
4,RO,11,Rondônia,Cerejeiras,1100056,61940,22170,192882,107819,384811,63713,448524,25010,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,2017,110005


**Município - População Estimada (IBGE)**

In [16]:
ibge_municipio_populacao_estimada = pd.read_parquet('../data/ibge_municipio_populacao_estimada.parquet')

# algumas conversoes e mapeamentos simples
ibge_municipio_populacao_estimada['ufCodigo'] = ibge_municipio_populacao_estimada['ufCodigo'].astype(str)
ibge_municipio_populacao_estimada['municipioCodigo'] = ibge_municipio_populacao_estimada['municipioCodigo'].astype(str)
ibge_municipio_populacao_estimada['municipioCodigo'] = ibge_municipio_populacao_estimada['municipioCodigo'].str.pad(5, fillchar='0')
ibge_municipio_populacao_estimada['municipioCodigo'] = (
    ibge_municipio_populacao_estimada['ufCodigo'].astype(str) + ibge_municipio_populacao_estimada['municipioCodigo']
)

ibge_municipio_populacao_estimada['municipioCodigo6d'] = ibge_municipio_populacao_estimada['municipioCodigo'].str.slice(0, 6)

# categorizacao da populacao estimada em faixas (baseado em fibonacci * 10k)
max_pop = int(ibge_municipio_populacao_estimada['populacaoEstimada'].max())
pop_bins = [i * 10000 for i in [0, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]] + [max_pop]
pop_labels = [
    ('{}:{}'.format(
        str(int(pop_bins[(i)])),
        str(int(pop_bins[(i+1)])) if (i + 1) < len(pop_bins) else str(max_pop)
    ))
    for i, n in enumerate(pop_bins[:-1])
]

ibge_municipio_populacao_estimada['faixaPopulacaoEstimada'] = pd.cut(
    ibge_municipio_populacao_estimada['populacaoEstimada'],
    bins=pop_bins,
    labels=pop_labels
)

inspect(ibge_municipio_populacao_estimada);

shape: (5570, 8)
columns: ['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'populacaoEstimada', 'ano', 'municipioCodigo6d', 'faixaPopulacaoEstimada']


,uf,ufCodigo,nomeMunicipio,municipioCodigo,populacaoEstimada,ano,municipioCodigo6d,faixaPopulacaoEstimada
0,RO,11,Alta Floresta D'Oeste,1100015,22945.000,2020,110001,20000:30000
1,RO,11,Ariquemes,1100023,107863.000,2020,110002,80000:130000
2,RO,11,Cabixi,1100031,5312.000,2020,110003,0:10000
3,RO,11,Cacoal,1100049,85359.000,2020,110004,80000:130000
4,RO,11,Cerejeiras,1100056,16323.000,2020,110005,10000:20000


**Município - Censo Básico (IBGE)**

In [17]:
ibge_setor_censo_basico = pd.read_parquet('../data/ibge_setor_censo_basico.parquet')
ibge_setor_censo_basico['codigoMunicipio'] = ibge_setor_censo_basico['codigoMunicipio'].astype(str)
# ibge_setor_censo_basico['codigoMunicipio6d'] = ibge_setor_censo_basico['codigoMunicipio'].str.slice(0, 6)
ibge_setor_censo_basico = (
    ibge_setor_censo_basico
    .assign(ano=lambda x: 2010)
    .drop(columns=['nomeUf', 'nomeMunicipio'])
    .rename(columns={'codigoUf': 'ufCodigo', 'siglaUf': 'uf', 'codigoMunicipio': 'municipioCodigo'})
    .merge(ibge_municipio, on=['uf', 'ufCodigo', 'municipioCodigo'], how='left')
)

ibge_municipio_censo_basico = (
    ibge_setor_censo_basico
    .groupby(['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'ano'])
    .agg(
        nomeGrandeRegiao=('nomeGrandeRegiao', 'first'),
        nomeMesoRegiao=('nomeMesoRegiao', 'first'),
        nomeMicroRegiao=('nomeMicroRegiao', 'first'),
        qtSetorCentiario=('codigoSetor', 'count'),
        qtDistritos=('codigoDistrito', 'count'),
        qtBairros=('codigoBairro', 'count'),
        qtDomicilios=('domicilios', 'sum'),
        # qtMoradores=('moradores', 'sum'), # deprecated: usar ibge_municipio_populacao_estimada

        mediaMoradoresDomicilios=('mediaMoradoresDomicilios', 'mean'),
        minMoradoresDomicilios=('mediaMoradoresDomicilios', 'min'),
        maxMoradoresDomicilios=('mediaMoradoresDomicilios', 'max'),

        mediaRendaMensalMoradores=('rendaMediaMensalMoradores', 'mean'),
        minRendaMensalMoradores=('rendaMediaMensalMoradores', 'min'),
        maxRendaMensalMoradores=('rendaMediaMensalMoradores', 'max'),

        qtSetorUrbano=('urbano', lambda x: x.astype(int).sum()),
        qtSetorRural=('rural', lambda x: x.astype(int).sum()),
        # qtSetorMetropolitano=('metropolitano', lambda x: x.astype(int).sum()) # TODO: variavel metropolitano foi exportada errada
    )
    .reset_index()
    .assign(partSetorUrbano=lambda x: x['qtSetorUrbano']/x['qtSetorCentiario'])
    .assign(partSetorRural=lambda x: x['qtSetorRural']/x['qtSetorCentiario'])
    #.assign(partSetorMetropolitano=lambda x: x['qtSetorRural']/x['qtSetorCentiario']) # TODO: fix metropolitano
    .assign(nivelUrbanizacao=lambda x: pd.cut(
        x=x['partSetorUrbano'],
        bins=[0, 0.2, 0.4, 0.6, 0.8, 1],
        labels=['muito baixo', 'baixo', 'moderado', 'alto', 'muito alto']
    ))
)

# inspect(ibge_setor_censo_basico);
inspect(ibge_municipio_censo_basico);

shape: (5565, 24)
columns: ['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'ano', 'nomeGrandeRegiao', 'nomeMesoRegiao', 'nomeMicroRegiao', 'qtSetorCentiario', 'qtDistritos', 'qtBairros', 'qtDomicilios', 'mediaMoradoresDomicilios', 'minMoradoresDomicilios', 'maxMoradoresDomicilios', 'mediaRendaMensalMoradores', 'minRendaMensalMoradores', 'maxRendaMensalMoradores', 'qtSetorUrbano', 'qtSetorRural', 'partSetorUrbano', 'partSetorRural', 'nivelUrbanizacao']


,uf,ufCodigo,nomeMunicipio,municipioCodigo,municipioCodigo6d,ano,nomeGrandeRegiao,nomeMesoRegiao,nomeMicroRegiao,qtSetorCentiario,qtDistritos,qtBairros,qtDomicilios,mediaMoradoresDomicilios,minMoradoresDomicilios,maxMoradoresDomicilios,mediaRendaMensalMoradores,minRendaMensalMoradores,maxRendaMensalMoradores,qtSetorUrbano,qtSetorRural,partSetorUrbano,partSetorRural,nivelUrbanizacao
0,AC,12,Acrelândia,1200013,120001,2010,Regio Norte,Vale do Acre,Rio Branco,16,16,16,3459.000,3.614,3.180,4.080,749.394,342.570,1761.270,5,11,0.312,0.688,baixo
1,AC,12,Assis Brasil,1200054,120005,2010,Regio Norte,Vale do Acre,Brasilia,10,10,10,1539.000,4.658,3.590,7.000,446.273,92.390,1126.380,3,7,0.300,0.700,baixo
2,AC,12,Brasiléia,1200104,120010,2010,Regio Norte,Vale do Acre,Brasilia,31,31,31,5930.000,3.926,2.970,6.500,626.717,210.740,1602.430,16,15,0.516,0.484,moderado
3,AC,12,Bujari,1200138,120013,2010,Regio Norte,Vale do Acre,Rio Branco,14,14,14,2330.000,3.600,3.000,4.260,813.732,496.640,1857.060,4,10,0.286,0.714,baixo
4,AC,12,Capixaba,1200179,120017,2010,Regio Norte,Vale do Acre,Rio Branco,18,18,18,2353.000,3.524,2.330,3.960,665.828,278.890,1230.180,5,13,0.278,0.722,baixo


**Pessoa Física - homicídios (SIM)**

In [18]:
sim_pf_homcidios = pd.read_csv(
    '../data/1. SIM (Homicidios)/DadosHomicidios_2013-18_SIM.csv',
    dtype={'dtnasc': str},
    low_memory=False
)

sim_pf_homcidios.columns = sim_pf_homcidios.columns.str.lower()

# Código relativo ao município onde ocorreu o óbito
sim_pf_homcidios['codmunocor'] = sim_pf_homcidios['codmunocor'].astype(str)

# combinando dados
sim_pf_homcidios = (
    sim_pf_homcidios
    .merge(ibge_municipio.rename(columns={'municipioCodigo6d': 'codmunocor'}), on='codmunocor')
    .merge(cbos.rename(columns={'codigoCbo': 'ocup'}), on='ocup', how='left')
)

# DE-PARA, conforme dicionario de dados do SIM
sim_pf_homcidios = sim_pf_homcidios.replace({
    # Tipo de morte violenta ou circunstâncias em que se deu a morte não  natural
    "circobito": {1: "acidente", 2: "suicídio", 3: "homicídio", 4: "outros", 9: "ignorado"},

    # Sexo do falecido
    "sexo": {0: "ignorado", 1: "masculino", 2: "feminino"},

    # Cor informada pelo responsável pelas informações do falecido.
    "racacor": {1: "Branca", 2: "Preta", 3: "Amarela", 4: "Parda", 5: "Indígena"},

    # Local de ocorrência do óbito.
    "lococor": {
        1: "hospital", 2: "outros estabelecimentos de saúde", 3: "domicílio",
        4: "via pública", 5: "outros", 6: "aldeia indígena", 9: "ignorado"
    },

    # Se refere ao atendimento médico continuado que o paciente recebeu,  ou não , durante a enfermidade que ocasionou o óbito.
    "assistmed": {1: "sim", 2: "não", 9: "ignorado"},
})

# Data em que occoreu o óbito
sim_pf_homcidios['dtobito'] = pd.to_datetime((sim_pf_homcidios['dtobito'].astype(int).astype(str)).str.pad(8, fillchar='0'), format='%d%m%Y')

# Extraindo outras variaveis a partir da data do óbito
sim_pf_homcidios = mg.extract_time_deltas(sim_pf_homcidios, 'dtobito')

# Horário do óbito.
sim_pf_homcidios['horaobito'] = sim_pf_homcidios['horaobito'].fillna(-9999)
sim_pf_homcidios['horaobito'] = (sim_pf_homcidios['horaobito'].astype(int).astype(str)).str.pad(4, fillchar='0')
sim_pf_homcidios['horaobito'] = (sim_pf_homcidios['horaobito'].str.slice(0, 2).astype(int) * 60 + sim_pf_homcidios['horaobito'].str.slice(2, 4).astype(int))/float(60)
sim_pf_homcidios['horaobito'] = np.where(sim_pf_homcidios['horaobito'] < 0, np.nan, sim_pf_homcidios['horaobito'])

# Tipo de trabalho que o falecido desenvolveu na maior parte de sua vida produtiva (CBO 2002).
sim_pf_homcidios['ocup'] = sim_pf_homcidios['ocup'].fillna(-1)
sim_pf_homcidios['ocup'] = sim_pf_homcidios['ocup'].astype(int)

# extraindo outras caracteristicas a partir da ocupacao
sim_pf_homcidios['ocupacaoCbo'] = sim_pf_homcidios['ocupacaoCbo'].fillna('IGNORADO')
sim_pf_homcidios['semOcupacao'] = sim_pf_homcidios['ocup'].isin([-1, 999991, 999994, 998999])

# O primeiro, de 1 dígito, indica a unidade da idade (se 1 = minuto, se 2 = hora, se 3 = mês, se 4 = ano,  se = 5 idade maior que 100 anos)
# Anos - subcampo varia de 00 a 99; - 9 - ignorado)
sim_pf_homcidios = sim_pf_homcidios[(sim_pf_homcidios['idade'] >= 400) & (sim_pf_homcidios['idade'] < 500)]
sim_pf_homcidios['idade'] = (sim_pf_homcidios['idade'].astype(str).str.slice(1, 3)).astype(int)

# Se refere ao atendimento médico continuado que o paciente recebeu, ou não, durante a enfermidade que ocasionou o óbito.
sim_pf_homcidios['assistmed'] = sim_pf_homcidios['assistmed'].fillna('não')

# População alvo de estudo
# considerando apenas homicídios
# Considerando apenas homicídios de Jovens (15 a 29 anos)
sim_pf_homcidios = sim_pf_homcidios[sim_pf_homcidios['circobito'] == 'homicídio']
sim_pf_homcidios = sim_pf_homcidios[(sim_pf_homcidios['idade'] >= 15) & (sim_pf_homcidios['idade'] < 30)]

# removendo variaveis com preenchimento muito baixo (<20%)
sim_pf_homcidios_fill_rate = mg.fill_rate(sim_pf_homcidios)
sim_pf_homcidios_low_fill_rate_cols = sim_pf_homcidios_fill_rate[sim_pf_homcidios_fill_rate['Fill (%)'] < 20]['Variable'].tolist()
sim_pf_homcidios_cols = [i for i in sim_pf_homcidios.columns.tolist() if i not in sim_pf_homcidios_low_fill_rate_cols]
sim_pf_homcidios = sim_pf_homcidios[sim_pf_homcidios_cols]

# removendo variaveis irrelevantes: 
sim_pf_homcidios_bad_cols = [
    'numerolote', 'linhac', 'tipobito', 'comunsvoim', 'dtrecebim', 'difdata',
    'stcodifica', 'versaosist', 'versaoscb', 'stdoepidem', 'stdonova', 'numerolote', 'dtrecoriga'
]

sim_pf_homcidios = sim_pf_homcidios.drop(columns=sim_pf_homcidios_bad_cols)

# TODO Campos relevantes (motivo do obito): LINHAA, CAUSABAS_O, necropsia, causabas, atestado, lococor
# estciv  (comunidade)
# dtatestado, assistmed, ATESTANTE (recuperacao)
# ESCFALAGR1, esc, esc2010, SERIESCFAL (ensino)
# fonte, dtinvestig, fonteinv, necropsia

print('low_fill_rate_cols: ', sim_pf_homcidios_low_fill_rate_cols)

inspect(sim_pf_homcidios);

low_fill_rate_cols:  ['nudiasinf', 'escmae2010', 'seriescmae', 'ocupmae', 'qtdfilvivo', 'qtdfilmort', 'gravidez', 'semagestac', 'gestacao', 'parto', 'obitoparto', 'peso', 'dtcadinf', 'morteparto', 'dtconcaso', 'cb_pre', 'escmaeagr1', 'escmae', 'idademae', 'estabdescr', 'altcausa', 'causamat', 'acidtrab', 'exame', 'cirurgia', 'tpresginfo', 'nudiasobin', 'fontes', 'dtregcart', 'numregcart', 'codmuncart', 'nudiasobco', 'tpnivelinv', 'tpmorteoco', 'obitopuerp', 'obitograv', 'dtconinv', 'dtcadinv', 'tpobitocor', 'linhaii', 'dtrecorig', 'fontesinf', 'linhad']
shape: (182444, 49)
columns: ['contador', 'origem', 'dtobito', 'horaobito', 'natural', 'dtnasc', 'idade', 'sexo', 'racacor', 'estciv', 'esc', 'esc2010', 'seriescfal', 'ocup', 'codmunres', 'lococor', 'codestab', 'codmunocor', 'assistmed', 'necropsia', 'linhaa', 'linhab', 'causabas', 'dtatestado', 'circobito', 'fonte', 'tppos', 'dtinvestig', 'causabas_o', 'dtcadastro', 'atestante', 'fonteinv', 'codmunnatu', 'codificado', 'atestado', 'escf

,contador,origem,dtobito,horaobito,natural,dtnasc,idade,sexo,racacor,estciv,esc,esc2010,seriescfal,ocup,codmunres,lococor,codestab,codmunocor,assistmed,necropsia,linhaa,linhab,causabas,dtatestado,circobito,fonte,tppos,dtinvestig,causabas_o,dtcadastro,atestante,fonteinv,codmunnatu,codificado,atestado,escfalagr1,uf,ufCodigo,nomeUf,nomeMunicipio,nomeMunicipioNorm,municipioCodigo,ocupacaoCbo,dtobito_yyyymm,dtobito_year,dtobito_quarter,dtobito_month,dtobito_day,semOcupacao
0,49,1,2013-06-30,5.000,NaN,9101987.000,25,masculino,Branca,5.000,3.000,2.000,NaN,-1,120035,hospital,2001594.000,120035,não,NaN,*X990,NaN,X990,NaN,homicídio,3.000,NaN,NaN,X990,14082013.000,NaN,NaN,NaN,NaN,NaN,NaN,AC,12,Acre,Marechal Thaumaturgo,MARECHAL THAUMATURGO,1200351,IGNORADO,201306,2013,2,6,30,True
2,1077,1,2015-10-22,17.000,812.000,4101987.000,28,feminino,Branca,2.000,2.000,1.000,NaN,999992,120035,domicílio,NaN,120035,não,2.000,*X940,NaN,X940,22102015.000,homicídio,1.000,N,NaN,X940,30102015.000,5.000,NaN,120035.000,S,X940,10.000,AC,12,Acre,Marechal Thaumaturgo,MARECHAL THAUMATURGO,1200351,DONA DE CASA,201510,2015,4,10,22,False
4,3385,1,2016-11-29,2.000,812.000,30081987.000,29,masculino,Branca,1.000,4.000,3.000,3.000,-1,120035,hospital,2001594.000,120035,sim,2.000,*R571,*X994,X994,29112016.000,homicídio,3.000,N,NaN,X994,16122016.000,5.000,NaN,120035.000,S,R571/X994,6.000,AC,12,Acre,Marechal Thaumaturgo,MARECHAL THAUMATURGO,1200351,IGNORADO,201611,2016,4,11,29,True
5,267,1,2013-04-28,17.083,NaN,29011991.000,22,feminino,Parda,5.000,3.000,1.000,4.000,999992,120030,outros,NaN,120030,não,2.000,*Y004,*S061,Y004,28042013.000,homicídio,2.000,N,NaN,Y004,23082013.000,5.000,NaN,NaN,NaN,NaN,NaN,AC,12,Acre,Feijó,FEIJO,1200302,DONA DE CASA,201304,2013,2,4,28,False
11,1867,1,2013-11-01,23.000,NaN,13081993.000,20,masculino,Parda,5.000,1.000,0.000,NaN,999992,120030,outros,NaN,120030,não,2.000,*S211,*S311,X998,2112013.000,homicídio,3.000,N,NaN,X998,4112013.000,5.000,NaN,NaN,NaN,NaN,NaN,AC,12,Acre,Feijó,FEIJO,1200302,DONA DE CASA,201311,2013,4,11,1,False


**Município - homicídios (SIM)**

In [19]:
print((sim_pf_homcidios['dtobito_year']).unique())

sim_municipio_homicidios = (
    sim_pf_homcidios
    # [(sim_pf_homcidios['uf'].isin(UFS)) & (sim_pf_homcidios['dtobito'].dt.year == 2018)]
    .rename(columns={'codmunocor': 'municipioCodigo6d'})
    .groupby(['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'dtobito_year'])
    .agg(
        qtHomicidios=('contador', 'count'),
        qtHomicidiosFeminino=('sexo', lambda x: (x == 'feminino').astype(int).sum()),
        qtHomicidiosMasculino=('sexo', lambda x: (x == 'masculino').astype(int).sum()),

        # TODO homicidios por racacor
        # TODO homicidios por lococor
        # TODO homicidios por grupo de CID

        qtHomicidiosComAssistmed=('assistmed', lambda x: (x == 'sim').astype(int).sum()),
        qtHomicidiosSemAssistmed=('assistmed', lambda x: (x == 'não').astype(int).sum()),
        qtHomicidiosSemOcupacao=('semOcupacao', lambda x: x.astype(int).sum()),
        qtHomicidiosComOcupacao=('semOcupacao', lambda x: (x == False).astype(int).sum()),
    )
    .reset_index()
)

inspect(sim_municipio_homicidios);

[2013 2015 2016 2014 2017 2018]
shape: (15893, 13)
columns: ['uf', 'ufCodigo', 'nomeMunicipio', 'municipioCodigo', 'municipioCodigo6d', 'dtobito_year', 'qtHomicidios', 'qtHomicidiosFeminino', 'qtHomicidiosMasculino', 'qtHomicidiosComAssistmed', 'qtHomicidiosSemAssistmed', 'qtHomicidiosSemOcupacao', 'qtHomicidiosComOcupacao']


,uf,ufCodigo,nomeMunicipio,municipioCodigo,municipioCodigo6d,dtobito_year,qtHomicidios,qtHomicidiosFeminino,qtHomicidiosMasculino,qtHomicidiosComAssistmed,qtHomicidiosSemAssistmed,qtHomicidiosSemOcupacao,qtHomicidiosComOcupacao
0,AC,12,Acrelândia,1200013,120001,2013,1,0,1,0,1,1,0
1,AC,12,Acrelândia,1200013,120001,2014,1,0,1,0,1,1,0
2,AC,12,Acrelândia,1200013,120001,2016,3,0,3,0,3,1,2
3,AC,12,Acrelândia,1200013,120001,2017,1,0,1,0,1,1,0
4,AC,12,Acrelândia,1200013,120001,2018,4,0,4,2,2,3,1


# Write some handled datasets

In [20]:
data_names = {
    'cbos': cbos,
    'ibge_municipio': ibge_municipio,
    'rais_caged_municipio_funcionarios': rais_caged_municipio_funcionarios,
    'rfb_municipio_cnaes': rfb_municipio_cnaes,
    'inep_municipio_rendimento_escolar': inep_municipio_rendimento_escolar,
    'ibge_municipio_pib': ibge_municipio_pib,
    'ibge_municipio_populacao_estimada': ibge_municipio_populacao_estimada,
    'ibge_municipio_censo_basico': ibge_municipio_censo_basico,
    'sim_pf_homcidios': sim_pf_homcidios,
    'sim_municipio_homicidios': sim_municipio_homicidios
}

In [21]:
for name, df in data_names.items():
    df.to_parquet(f'../data/handled/{name}.parquet')